##Objective



1. Exploratory analysis of COVID 19 cases  for four different nations in the United Kingdom.
2. Data Visualization of the projected patterns for analysis.
3. Comparing the Covid cases in the United Kingdoms with cases in other countries such as Korean and India.

In [0]:
import pyspark.sql.functions as F 
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import plotly.express as ex

## Data preprocessing 

In [0]:
df_UnitedKingdom= spark.read.csv('/FileStore/tables/UK_Devolved_Nations_COVID19_2021_Dataset-3.csv', header = True, inferSchema = True )
df_PopulationData = spark.read.csv('/FileStore/tables/Populations_for_UK_and_Devolved_Nations.csv', header = True, inferSchema = True )

In [0]:
df_UnitedKingdom = df_UnitedKingdom.na.drop()
df_UnitedKingdom.show()

+----------+--------+---------+-----------------+----------+---------------+
|      date|areaName|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|
+----------+--------+---------+-----------------+----------+---------------+
|2020-03-26|   Wales|      113|              203|        36|              6|
|2020-03-27|   Wales|      180|              243|        37|              6|
|2020-03-28|   Wales|      172|              310|        53|              4|
|2020-03-29|   Wales|      148|              329|        71|             10|
|2020-04-19|   Wales|      334|              828|       153|             41|
|2020-04-20|   Wales|      276|              853|       150|              9|
|2020-04-21|   Wales|      304|              838|       135|             25|
|2020-04-22|   Wales|      274|              773|       132|             15|
|2020-04-23|   Wales|      234|              741|       123|             17|
|2020-04-24|   Wales|      243|              743|       119|            110|

In [0]:
df_UnitedKingdom = df_UnitedKingdom.withColumnRenamed("areaName", "Nation")
df_UnitedKingdom.show(5)

+----------+------+---------+-----------------+----------+---------------+
|      date|Nation|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|
+----------+------+---------+-----------------+----------+---------------+
|2020-03-26| Wales|      113|              203|        36|              6|
|2020-03-27| Wales|      180|              243|        37|              6|
|2020-03-28| Wales|      172|              310|        53|              4|
|2020-03-29| Wales|      148|              329|        71|             10|
|2020-04-19| Wales|      334|              828|       153|             41|
+----------+------+---------+-----------------+----------+---------------+
only showing top 5 rows



In [0]:
df_UnitedKingdom = df_PopulationData.join(df_UnitedKingdom, on = 'Nation', how = 'inner')
df_UnitedKingdom.show(5)

+------+---+------------------+----------+---------+-----------------+----------+---------------+
|Nation|_c0|        Population|      date|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|
+------+---+------------------+----------+---------+-----------------+----------+---------------+
| Wales|  2|3169607.2851451337|2020-03-26|      113|              203|        36|              6|
| Wales|  2|3169607.2851451337|2020-03-27|      180|              243|        37|              6|
| Wales|  2|3169607.2851451337|2020-03-28|      172|              310|        53|              4|
| Wales|  2|3169607.2851451337|2020-03-29|      148|              329|        71|             10|
| Wales|  2|3169607.2851451337|2020-04-19|      334|              828|       153|             41|
+------+---+------------------+----------+---------+-----------------+----------+---------------+
only showing top 5 rows



In [0]:
df_UnitedKingdom.na.drop()
df_UnitedKingdom = df_UnitedKingdom.drop("_c0")
df_UnitedKingdom.show(5)

+------+------------------+----------+---------+-----------------+----------+---------------+
|Nation|        Population|      date|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|
+------+------------------+----------+---------+-----------------+----------+---------------+
| Wales|3169607.2851451337|2020-03-26|      113|              203|        36|              6|
| Wales|3169607.2851451337|2020-03-27|      180|              243|        37|              6|
| Wales|3169607.2851451337|2020-03-28|      172|              310|        53|              4|
| Wales|3169607.2851451337|2020-03-29|      148|              329|        71|             10|
| Wales|3169607.2851451337|2020-04-19|      334|              828|       153|             41|
+------+------------------+----------+---------+-----------------+----------+---------------+
only showing top 5 rows



In [0]:
## Total active cases
df_UnitedKingdom = df_UnitedKingdom.withColumn('ActiveCases', ((df_UnitedKingdom["MildCases"] + df_UnitedKingdom["HospitalisedCases"])-df_UnitedKingdom["CasesCured"]-df_UnitedKingdom["ConfirmedDeaths"]))
df_UnitedKingdom.show(10)
display(df_UnitedKingdom)

+------+------------------+----------+---------+-----------------+----------+---------------+-----------+
|Nation|        Population|      date|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|ActiveCases|
+------+------------------+----------+---------+-----------------+----------+---------------+-----------+
| Wales|3169607.2851451337|2020-03-26|      113|              203|        36|              6|        274|
| Wales|3169607.2851451337|2020-03-27|      180|              243|        37|              6|        380|
| Wales|3169607.2851451337|2020-03-28|      172|              310|        53|              4|        425|
| Wales|3169607.2851451337|2020-03-29|      148|              329|        71|             10|        396|
| Wales|3169607.2851451337|2020-04-19|      334|              828|       153|             41|        968|
| Wales|3169607.2851451337|2020-04-20|      276|              853|       150|              9|        970|
| Wales|3169607.2851451337|2020-04-21|      30

Nation,Population,date,MildCases,HospitalisedCases,CasesCured,ConfirmedDeaths,ActiveCases
Wales,3169607.2851451337,2020-03-26,113,203,36,6,274
Wales,3169607.2851451337,2020-03-27,180,243,37,6,380
Wales,3169607.2851451337,2020-03-28,172,310,53,4,425
Wales,3169607.2851451337,2020-03-29,148,329,71,10,396
Wales,3169607.2851451337,2020-04-19,334,828,153,41,968
Wales,3169607.2851451337,2020-04-20,276,853,150,9,970
Wales,3169607.2851451337,2020-04-21,304,838,135,25,982
Wales,3169607.2851451337,2020-04-22,274,773,132,15,900
Wales,3169607.2851451337,2020-04-23,234,741,123,17,835
Wales,3169607.2851451337,2020-04-24,243,743,119,110,757


In [0]:
## Rearranging columns
df_UnitedKingdom = df_UnitedKingdom.select("date","Nation","Population","MildCases","HospitalisedCases","CasesCured","ConfirmedDeaths","ActiveCases")

In [0]:
df_UnitedKingdom.groupBy("Nation").sum("MildCases", "HospitalisedCases").show()

+----------------+--------------+----------------------+
|          Nation|sum(MildCases)|sum(HospitalisedCases)|
+----------------+--------------+----------------------+
|           Wales|        494264|                280843|
|         England|       4608297|               2840563|
|        Scotland|        377049|                246445|
|Northern Ireland|        191121|                125821|
+----------------+--------------+----------------------+



In [0]:
## Cases in Wales
df_wales = df_UnitedKingdom.filter(df_UnitedKingdom["Nation"]=="Wales")
df_wales.show(5)

+----------+------+------------------+---------+-----------------+----------+---------------+-----------+
|      date|Nation|        Population|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|ActiveCases|
+----------+------+------------------+---------+-----------------+----------+---------------+-----------+
|2020-03-26| Wales|3169607.2851451337|      113|              203|        36|              6|        274|
|2020-03-27| Wales|3169607.2851451337|      180|              243|        37|              6|        380|
|2020-03-28| Wales|3169607.2851451337|      172|              310|        53|              4|        425|
|2020-03-29| Wales|3169607.2851451337|      148|              329|        71|             10|        396|
|2020-04-19| Wales|3169607.2851451337|      334|              828|       153|             41|        968|
+----------+------+------------------+---------+-----------------+----------+---------------+-----------+
only showing top 5 rows



In [0]:
## Cases in Scotland
df_scotland = df_UnitedKingdom.filter(df_UnitedKingdom["Nation"]=="Scotland")
df_scotland.show(5)

+----------+--------+-----------------+---------+-----------------+----------+---------------+-----------+
|      date|  Nation|       Population|MildCases|HospitalisedCases|CasesCured|ConfirmedDeaths|ActiveCases|
+----------+--------+-----------------+---------+-----------------+----------+---------------+-----------+
|2020-03-27|Scotland|5466257.668711656|      165|              404|        72|              8|        489|
|2020-03-28|Scotland|5466257.668711656|      186|              511|        81|              7|        609|
|2020-03-29|Scotland|5466257.668711656|      139|              565|        95|              1|        608|
|2020-03-30|Scotland|5466257.668711656|      179|              627|       108|              6|        692|
|2020-03-31|Scotland|5466257.668711656|      430|              752|       135|             22|       1025|
+----------+--------+-----------------+---------+-----------------+----------+---------------+-----------+
only showing top 5 rows



In [0]:
## Total cases in scotland
df_UnitedKingdom.filter((df_UnitedKingdom['Nation'] == "Scotland") & (df_UnitedKingdom['MildCases'] > 0) & (df_UnitedKingdom['HospitalisedCases'] > 0)).count()

Out[12]: 426

In [0]:
## Total cases in Wales
df_UnitedKingdom.filter((df_UnitedKingdom['Nation'] == "Wales") & (df_UnitedKingdom['MildCases'] > 0) & (df_UnitedKingdom['HospitalisedCases'] > 0)).count()

Out[13]: 476

In [0]:
df_uk_latest_cases=df_UnitedKingdom.groupBy("date").sum().sort("date",ascending=True)
df_uk_latest_cases.select('date','sum(MildCases)', 'sum(HospitalisedCases)').toPandas().style.background_gradient(cmap='Reds')

,date,sum(MildCases),sum(HospitalisedCases)
0,2020-03-26,113,203
1,2020-03-27,345,647
2,2020-03-28,358,821
3,2020-03-29,287,894
4,2020-03-30,179,627
5,2020-03-31,430,752
6,2020-04-01,317,815
7,2020-04-02,3960,13350
8,2020-04-03,4105,14978
9,2020-04-04,3348,16906


In [0]:
df_uk_latest_cases=df_UnitedKingdom.groupBy("Nation", "date").sum().sort("sum(ConfirmedDeaths)", ascending=False)
df_uk_latest_cases.select("date", "Nation",'sum(MildCases)','sum(HospitalisedCases)','sum(CasesCured)','sum(ConfirmedDeaths)','sum(ActiveCases)').toPandas().style.background_gradient(cmap='Reds')
df_uk_latest_cases = df_uk_latest_cases.toPandas()
plot = df_uk_latest_cases.melt(id_vars="date", value_vars=['sum(CasesCured)',"sum(ConfirmedDeaths)","sum(ActiveCases)"])
matt = ex.treemap(plot, path=['variable'], values="value", height=500, width=800,
                color_discrete_sequence=["red","blue","green"])
matt.show() 


/databricks/python/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## Death cases in all 4 regions

In [0]:
df_state_death_cases = df_UnitedKingdom.filter(df_UnitedKingdom["ConfirmedDeaths"]>0)
df_state_death_cases.sort(df_state_death_cases["ConfirmedDeaths"].desc())
display(df_state_death_cases)

date,Nation,Population,MildCases,HospitalisedCases,CasesCured,ConfirmedDeaths,ActiveCases
2020-03-26,Wales,3169607.2851451337,113,203,36,6,274
2020-03-27,Wales,3169607.2851451337,180,243,37,6,380
2020-03-28,Wales,3169607.2851451337,172,310,53,4,425
2020-03-29,Wales,3169607.2851451337,148,329,71,10,396
2020-04-19,Wales,3169607.2851451337,334,828,153,41,968
2020-04-20,Wales,3169607.2851451337,276,853,150,9,970
2020-04-21,Wales,3169607.2851451337,304,838,135,25,982
2020-04-22,Wales,3169607.2851451337,274,773,132,15,900
2020-04-23,Wales,3169607.2851451337,234,741,123,17,835
2020-04-24,Wales,3169607.2851451337,243,743,119,110,757


Databricks visualization. Run in Databricks to view.

##Confirmed, Active, Death Cases over a period of 2 years

In [0]:
graph = df_UnitedKingdom.groupby('date').sum().toPandas()
graph = graph.melt(id_vars='date', value_vars=['sum(CasesCured)', 'sum(ConfirmedDeaths)', 'sum(ActiveCases)'],
         var_name='Case', value_name='No. of Cases')
graph.head()
fig=ex.area(graph, x='date', y='No. of Cases', color='Case', title = 'Cases over a period of 2 years', color_discrete_sequence=["red", "blue", "green"])
fig.show()

In [0]:
df_UnitedKingdom = df_UnitedKingdom.withColumn('TotalConfirmed', ((df_UnitedKingdom["MildCases"] + df_UnitedKingdom["HospitalisedCases"])))
display(df_UnitedKingdom)

date,Nation,Population,MildCases,HospitalisedCases,CasesCured,ConfirmedDeaths,ActiveCases,TotalConfirmed
2020-03-26,Wales,3169607.2851451337,113,203,36,6,274,316
2020-03-27,Wales,3169607.2851451337,180,243,37,6,380,423
2020-03-28,Wales,3169607.2851451337,172,310,53,4,425,482
2020-03-29,Wales,3169607.2851451337,148,329,71,10,396,477
2020-04-19,Wales,3169607.2851451337,334,828,153,41,968,1162
2020-04-20,Wales,3169607.2851451337,276,853,150,9,970,1129
2020-04-21,Wales,3169607.2851451337,304,838,135,25,982,1142
2020-04-22,Wales,3169607.2851451337,274,773,132,15,900,1047
2020-04-23,Wales,3169607.2851451337,234,741,123,17,835,975
2020-04-24,Wales,3169607.2851451337,243,743,119,110,757,986


In [0]:
df_latest = df_UnitedKingdom.groupBy(df_UnitedKingdom["Nation"], df_UnitedKingdom["TotalConfirmed"], df_UnitedKingdom["ConfirmedDeaths"], df_UnitedKingdom["CasesCured"], df_UnitedKingdom["ActiveCases"]).agg({"date": "max"})
df_latest=df_latest.groupBy("Nation").sum()
df_latest.show()

+----------------+-------------------+--------------------+---------------+----------------+
|          Nation|sum(TotalConfirmed)|sum(ConfirmedDeaths)|sum(CasesCured)|sum(ActiveCases)|
+----------------+-------------------+--------------------+---------------+----------------+
|           Wales|             775005|                5449|          22224|          747332|
|         England|            7448860|               74943|         347512|         7026405|
|        Scotland|             622050|                5189|          20704|          596157|
|Northern Ireland|             316524|                2146|           7538|          306840|
+----------------+-------------------+--------------------+---------------+----------------+

